In [14]:
import os 
import glob
import pandas as pd

In [2]:
def path_to_expID(path:str):
    ID = {}
    filter_background = os.path.basename(path).split('_')[-1].replace('.tsv.gz', '')
    filter_foreground = os.path.basename(path).split('_')[-2]
    #print(filter_background, filter_foreground)
    # Extract Values
    ID['filter_background_reads'] = filter_background.split('lim')[-1].split('Across')[0]
    ID['filter_background_samples'] = filter_background.split('Across')[-1]
    ID['filter_background_cohort'] = filter_background.split('Normals')[1].split('Cohort')[0]
    ID['filter_foreground_reads'] = filter_foreground.split('Lim')[-1].split('Across')[0]
    ID['filter_foreground_samples'] = filter_foreground.split('Across')[-1]
    ID['filter_foreground_target'] = (filter_foreground.split('Lim')[1].replace('SampleLim', '').replace('Cohort', ''))
    return ID
    

def expID_to_block(ID_split: dict):
    ID_dict = {'Any': 'A', 'None': 'N', '10': 'X', 'paired': 'P', 'Gtex': 'G', 'Core_GTEx': 'R', 'Full': 'F'}
    motif = 'A' # Any
    for k, v in ID_split.items():
        try:
            assert(v != '10') #replace roman letter
            ID_split[k] = str(int(float(v)))
        except:
            ID_split[k] = ID_dict[v]

    return ID_split['filter_foreground_target'] + \
    ID_split['filter_foreground_reads'] + \
    ID_split['filter_foreground_samples'] + \
    ID_split['filter_background_reads'] + \
    ID_split['filter_background_samples'] + \
    ID_split['filter_background_cohort'] + \
    motif 
    

In [21]:
### Inputs ###

basepath = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102' #TCGA_Breast_1102'
filtering_run = 'filtering_samples/filters_19May_order_5ge_wAnnot_GPstar'
dir_path = os.path.join(basepath, filtering_run)
fasta_paths = glob.glob(os.path.join(dir_path, '*fa*'))
path_filtered = glob.glob(os.path.join(dir_path, '*Filt*'))

In [22]:
res_list = []
for filt_file in path_filtered:
    ID_split = path_to_expID(filt_file)
    ID = expID_to_block(ID_split)
    renamed = '_'.join(os.path.basename(filt_file).split('_')[0:2]) + '_'+ ID + '.tsv.gz'
    res_list.append([filt_file, renamed])
    

In [23]:
output_file = os.path.join(dir_path, 'tmp_rename.txt')

res = pd.DataFrame(res_list)
res.to_csv(output_file, index=None, header=False, sep = '\t')

print(f'save to {output_file}')

save to /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/tmp_rename.txt
